## Deliverable 2. Create a Customer Travel Destinations Map.

In [14]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,East London,-33.02,27.91,69.80,88,75,14.99,ZA,2020-11-28 17:23:20,broken clouds
1,1,Roald,62.58,6.13,44.60,65,20,10.29,NO,2020-11-28 17:23:20,few clouds
2,2,Clyde River,70.47,-68.59,-4.00,83,75,16.11,CA,2020-11-28 17:23:20,light snow
3,3,San Juan,-31.54,-68.54,89.69,30,1,15.08,AR,2020-11-28 17:23:20,clear sky
4,4,Quatre Cocos,-20.21,57.76,77.00,78,75,11.41,MU,2020-11-28 17:23:20,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
Temp_Min = float(input("What is the minimum temperature you would like for your trip? "))
Temp_Max = float(input("What is the maximum temperature you would like for your trip? "))

# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= Temp_Max) & \
                                       (city_data_df["Max Temp"] >= Temp_Min)]
preferred_cities_df.head(10)

What is the minimum temperature you would like for your trip? 5
What is the maximum temperature you would like for your trip? 75


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,East London,-33.02,27.91,69.80,88,75,14.99,ZA,2020-11-28 17:23:20,broken clouds
1,1,Roald,62.58,6.13,44.60,65,20,10.29,NO,2020-11-28 17:23:20,few clouds
5,5,Jamestown,42.10,-79.24,39.00,69,75,13.87,US,2020-11-28 17:23:20,broken clouds
6,6,Copiapo,-27.37,-70.33,69.80,52,0,8.05,CL,2020-11-28 17:23:14,clear sky
8,8,Izhma,65.01,53.91,15.01,94,94,4.97,RU,2020-11-28 17:23:21,overcast clouds
9,9,Ribeira Grande,38.52,-28.70,57.20,67,40,6.93,PT,2020-11-28 17:23:21,scattered clouds
14,14,Yarmouth,41.71,-70.23,55.40,54,20,5.82,US,2020-11-28 17:23:21,few clouds
16,16,Ushuaia,-54.80,-68.30,42.80,75,40,14.99,AR,2020-11-28 17:20:38,scattered clouds
17,17,Codrington,-38.27,141.97,55.00,91,100,18.21,AU,2020-11-28 17:23:22,light rain
18,18,Hermanus,-34.42,19.23,66.00,67,10,3.00,ZA,2020-11-28 17:23:22,clear sky


In [4]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Country        6
Date           0
Description    0
dtype: int64

In [5]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,East London,-33.02,27.91,69.80,88,75,14.99,ZA,2020-11-28 17:23:20,broken clouds
1,1,Roald,62.58,6.13,44.60,65,20,10.29,NO,2020-11-28 17:23:20,few clouds
5,5,Jamestown,42.10,-79.24,39.00,69,75,13.87,US,2020-11-28 17:23:20,broken clouds
6,6,Copiapo,-27.37,-70.33,69.80,52,0,8.05,CL,2020-11-28 17:23:14,clear sky
8,8,Izhma,65.01,53.91,15.01,94,94,4.97,RU,2020-11-28 17:23:21,overcast clouds


In [6]:
clean_df = clean_df.rename(columns={'Description': 'Current Description'})
clean_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,East London,-33.02,27.91,69.80,88,75,14.99,ZA,2020-11-28 17:23:20,broken clouds
1,1,Roald,62.58,6.13,44.60,65,20,10.29,NO,2020-11-28 17:23:20,few clouds
5,5,Jamestown,42.10,-79.24,39.00,69,75,13.87,US,2020-11-28 17:23:20,broken clouds
6,6,Copiapo,-27.37,-70.33,69.80,52,0,8.05,CL,2020-11-28 17:23:14,clear sky
8,8,Izhma,65.01,53.91,15.01,94,94,4.97,RU,2020-11-28 17:23:21,overcast clouds


In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,East London,ZA,69.80,broken clouds,-33.02,27.91,NaN
1,Roald,NO,44.60,few clouds,62.58,6.13,NaN
5,Jamestown,US,39.00,broken clouds,42.10,-79.24,NaN
6,Copiapo,CL,69.80,clear sky,-27.37,-70.33,NaN
8,Izhma,RU,15.01,overcast clouds,65.01,53.91,NaN
9,Ribeira Grande,PT,57.20,scattered clouds,38.52,-28.70,NaN
14,Yarmouth,US,55.40,few clouds,41.71,-70.23,NaN
16,Ushuaia,AR,42.80,scattered clouds,-54.80,-68.30,NaN
17,Codrington,AU,55.00,light rain,-38.27,141.97,NaN
18,Hermanus,ZA,66.00,clear sky,-34.42,19.23,NaN


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
        # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df     

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,East London,ZA,69.80,broken clouds,-33.02,27.91,Tu Casa
1,Roald,NO,44.60,few clouds,62.58,6.13,Flem Cabins
5,Jamestown,US,39.00,broken clouds,42.10,-79.24,Hampton Inn & Suites Jamestown
6,Copiapo,CL,69.80,clear sky,-27.37,-70.33,Antay Casino Hotel
8,Izhma,RU,15.01,overcast clouds,65.01,53.91,NaN
...,...,...,...,...,...,...,...
706,Kathmandu,NP,48.20,few clouds,27.72,85.32,Hotel Shanker Kathmandu
709,Hammerfest,NO,41.00,light rain,70.66,23.68,Skytterhuset
710,Barkot,IN,51.62,few clouds,30.82,78.20,Camp Nirvana Yamunotri
713,Talbahat,IN,59.70,clear sky,25.05,78.43,PWD Guest House


In [20]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()
# hotel_df.count()
clean_hotel_df = hotel_df.dropna()

clean_hotel_df.isnull().sum()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,East London,ZA,69.80,broken clouds,-33.02,27.91,Tu Casa
1,Roald,NO,44.60,few clouds,62.58,6.13,Flem Cabins
5,Jamestown,US,39.00,broken clouds,42.10,-79.24,Hampton Inn & Suites Jamestown
6,Copiapo,CL,69.80,clear sky,-27.37,-70.33,Antay Casino Hotel
9,Ribeira Grande,PT,57.20,scattered clouds,38.52,-28.70,Quinta da Meia Eira
...,...,...,...,...,...,...,...
706,Kathmandu,NP,48.20,few clouds,27.72,85.32,Hotel Shanker Kathmandu
709,Hammerfest,NO,41.00,light rain,70.66,23.68,Skytterhuset
710,Barkot,IN,51.62,few clouds,30.82,78.20,Camp Nirvana Yamunotri
713,Talbahat,IN,59.70,clear sky,25.05,78.43,PWD Guest House


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")


In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}<Current Description/dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))